![title](https://miro.medium.com/max/400/1*2kYdgcRC9yn0Arr5vjAWeg.png)

***

# Azure Logic Apps

Mit den Azure logic Apps können wir verschiedene Services oder Programmabläufe in einem Workflow anordnen und ausführen lassen. Die Ausführung kann dabei durch verschiedene Trigger ausgelöst werden.

- Http Trigger
- Blob Storage Trigger
- TimerTrigger 
- CosmosDBTrigger
- QueueTrigger 
- EventHubTrigger 
- ServiceBusQueueTrigger
- ServiceBusTopicTrigger

Wir werden den BlobStorage Trigger verwenden, um die Logic App auszulösen wenn ein Bild in unseren Blob Storage geschrieben wird. Anschließend wollen wir eine Vorhersage durch unser erstelltes Custom Vision Modell ausführen. Da wir prozesse auch parallelisieren können wir gleichzeitig eine Vorhersage des Computer Vision Services auf das hochgeladene Bild anwenden. Anschließend müssen wir noch die JSON Rückgaben bereinigen und die Ergebnisse der beiden Modelle zusammenführen. 

Das schöne ist, dass wir die Logik App und alle Verbindungen über das Azure Portal konfigurieren können.

Mit den folgenden Schritten können wir die Logik App erstellen.

1. Über das Azure Portal auf die Logik App zugreifen
2. Erstellen eine Bolb Storage Triggers
3. Implementieren einer Vorhersage des Custom Vison Models
4. Testen der Logik App
5. Implementieren des Computer Vision OCR Services
6. Prasen der Json ausgaben
7. Zusammen führen der Json ausgaben und Speichern der Ergebnisse

Hier fndest du noch weiter Informationen zu den Azure Logik Apps

- [Übersicht: Was ist Azure Logic Apps?](https://docs.microsoft.com/de-de/azure/logic-apps/logic-apps-overview)
- [Schnellstart: Erstellen Ihres ersten automatisierten Workflows mit Azure Logic Apps – Azure-Portal](https://docs.microsoft.com/de-de/azure/logic-apps/quickstart-create-first-logic-app-workflow)
- [Erstellen Ihrer ersten Funktion mit Visual Studio](https://docs.microsoft.com/de-de/azure/azure-functions/functions-create-your-first-function-visual-studio)
- [Erstellen Ihrer ersten Funktion mit Visual Studio Code](https://docs.microsoft.com/de-de/azure/azure-functions/functions-create-first-function-vs-code)
- [Schnellstart: Erstellen und Veröffentlichen einer Funktion in Azure Functions mithilfe von Java](https://docs.microsoft.com/de-de/azure/azure-functions/functions-create-first-java-maven)
- [Erstellen Ihrer ersten PowerShell-Funktion in Azure (Vorschauversion)](https://docs.microsoft.com/de-de/azure/azure-functions/functions-create-first-function-powershell)

- [Erstellen einer durch HTTP ausgelösten Funktion in Azure](https://docs.microsoft.com/de-de/azure/azure-functions/functions-create-first-function-python)


***

In [5]:
## <font color='#0066cc'></font>

<img src="../media/CustomVisionImages/CustomVisionCreateProject.gif" width="1000">


SyntaxError: invalid syntax (<ipython-input-5-f04ccbf367cf>, line 3)

## <font color='#0066cc'>Erstellen eine Bolb Storage Triggers</font>


Name = blostorage container!!!!

Navigiere über das Azure Portal in deine in die Erstellte Resourcen gruppe und wähle die Azure Logic App. Auf der Übersicht Seite der Logic App wirst du ein Infromationsvideo zu dem Service sowie eine Auswahl von verschiedenen Triggern entdecken.


- Wähle die Erstellund einer Leeren Logic App Vorlage
- Gebe in des Suchfeld "BlobStorage" ein ein wähle einen Verbindungsnamen sowie einen BlobStorage mit dem du dich verbinden möchtest
- Wähle die Zugriffsrate auf den BlobStorage (zum testen habe ich jeden Minute gewählt) und klicke auf "Next Step" um den Trigger zu erstellen
- Speichere den Logic App Workflow bevor du zum nächsten SChritt weiter gehst

<img src="../media/LogicAppImages/BlobTrigger.gif" width="1000">

***

## <font color='#0066cc'>Implementieren einer Vorhersage des Custom Vison Models</font>

Um das Custom Vison Modell in unseres Workflow zu integrieren müssen wir zunächst einen Connector zu der Prediction API erstellen. Dies wir automatisch abgefrag wenn wir in unseren Workflow ein Custom Vision element einfügen. Anschließend müssen wir unser Projekt und das bereitgestellte Modell, sowie den Bild path definieren, um eine Vorhersage zu erhalten

<img src="../media/LogicAppImages/CustomVision.gif" width="1000">

***

## <font color='#0066cc'>Testen der Logik App</font>

Wir können die Logik App auf zwei wege teste. Zunächst müssen wir die Logik App starten. Danach können wir entweder die bereits erstellte PowerApp nutzen um die Bilder Hochzuladen oder wir laden die Bilder direkt in den BlobStorage.
Da ich es hier besser darstellen kann, zeige ich euch den Test mit dem direkten Upload in den BlobStorage. Ich würde an euerer Stelle aber beide Varianten testen, um sicher zu gehen das alle Schritte funktionieren.

Falls ihr Probleme mit der Custm Vison Schnittstelle habt schaut einfach mal, ob die Zugriffskontrolle eures Bobstorages richtig konfiguriert ist.


<img src="../media/LogicAppImages/CustomVisionTest.gif" width="1000">

***

## <font color='#0066cc'>Implementieren des Computer Vision OCR Services</font>

Für die Erstellung und die Auswahl des richtigen Computer Vision Services ist ein wenig vorwissen notwending. 

Microsoft bietet grundsätzlich 2 verschiedene OCR Modelle an. Das OCR Modell ist für die Erkennung von Computergeschrieben Texten und das recognizeText Text Modell kann für die Erkennung von Handgeschriebenen texten genutzt werden. Wir werden das letztere für unsere Vorhersagen nutzen, da das OCR Model nur sehr schlechte Ergebnisse liefert. Dies kann verschiedene Gründe haben

- Wölbung auf der Drehscheide der Zahlen verzerrt die die Zahlen 
- Es die auf den zählern dargestellten Fonts weichen zu start von den Trainingsdaten ab
- Schlecht Qualität der Bilder 
- Andere Font größe der dargestellten Bilder

Nach mehreren Test habe ich aber festgestellt das die recognizeText also die Handschrifterkennung besser zu unseren Daten passt und wir diese Nutzen können.


Dies Führt allerdings dazu das wir nicht die Vorgefertigten LogikApp Templates für Computer Vision nehmen könnne, da die recognizeText Funktion noch nicht integriert ist. Dies können wir aber einfach umgehen indem wir ein HTTP Template nutzen und den Service zunächst mt einem POST und dannach mit einem GET request ansprechen. Nachdem wir unsere Zugangsschlüssen zu dem Service gesendet haben, können wir mit dem GET request die Vohersage abfragen.



### <font color='#0066cc'>POST request Computer Vision OCR Services</font>


Dazu müssen wir unserem Endpoint und in dem Queries unserer Anfrage zusätzlich parameter mitgeben.


Werte für die Definition der HTTP Anfrage:

URI:      Comuter Vision ENDPOINT + /vision/v2.0/recognizeText                              

|          |      key                       |  value                                          |
|----------|--------------------------------|-------------------------------------------------|
| Header:  |    OCP-Apim-Subscription-Key   |   your-Computer-Vision-Key                      |
| Queries: |    mode                        |   Handwritten                                   |


Body:   
{                                                                                 
            "url": " your-storage-account-url @{triggerBody()?['Path']}"}                            
            } 

Um ein besseres Verständis zu den Computer Vision Services zu erhalten und die HTTP anfragen kennzulernen kannst du in dem folgenen Link einige [Cognetive Services Testen](https://azure.microsoft.com/de-de/services/cognitive-services/computer-vision/).

Die HTTP Anfragen und deren Parameter kannst du dir [hier](https://westus.dev.cognitive.microsoft.com/docs/services/5adf991815e1060e6355ad44/operations/56f91f2e778daf14a499e1fa) anschauen.

<img src="../media/LogicAppImages/ComputerVision.gif" width="1000">

### <font color='#0066cc'>Einbauen eines Delays in die Logic APP</font>

Da wir im nächsten Schritt auf den Header zugreifen müssen um den Operation-Location key von dem Service zu erhalten, müssen wir einen Delay einbauen, um auf eine Antwort von dem Server zu warten.

<img src="../media/LogicAppImages/AddDelay.gif" width="1000">

### <font color='#0066cc'>Einbauen einer Condition</font>

Neben dem Deploy können wir auch noch eine Condition einbauen, um unsere Logic APP zu beenden und evlt. eine Email zu senden das der Server nicht mehr errichbar ist

<img src="../media/LogicAppImages/AddCondition.gif" width="1000">

### <font color='#0066cc'>Parsen der Header aus der POST Request Rückgabe</font>

Im nächsten schritt müssen wir nur noch den Header von der Rückgabe des POST Requests Parsen damit wir auf den Operation-Location key in der GET Abfrage zugreifen können. Ihr könnt die Logik APP einfach einmal laufen lassen und euch die Ausgabe aus dem POST request anschauen bzw. kopieren und müsst dann nur noch die Datentypen für das Shema definieren.

Oder du nimmst einfach das Shema das ich dir nach dem GIF bereitstelle. ;D

<img src="../media/LogicAppImages/AddParseJson.gif" width="1000">

{
    "Cache-Control": {
        "type": "string"
    },
    "Content-Length": {
        "type": "string"
    },
    "Date": {
        "type": "string"
    },
    "Expires": {
        "type": "string"
    },
    "Operation-Location": {
        "type": "string"
    },
    "Pragma": {
        "type": "string"
    },
    "Strict-Transport-Security": {
        "type": "string"
    },
    "X-AspNet-Version": {
        "type": "string"
    },
    "X-Powered-By": {
        "type": "string"
    },
    "apim-request-id": {
        "type": "string"
    },
    "x-content-type-options": {
        "type": "string"
    }
}

### <font color='#0066cc'>Abfragen der Modellausgaben durch einen GET Request</font>

Nun können wir die Rükgabe aus dem vorherigen POST Request nutzen und auf die Uniform Resource Identifier (URI) aus der Operation-Location zugreifen. Mit einem GET Abfragen werden wir nun auf diese Resource zugreifen und nach der Modellausgabe fragen.

Da dies eine eher kreative Lösung ist (weil Microsoft die Handschrifterkennung für die Logik APps leider noch nicht implementiert hat) müssen wir den Zugriff auf die Operation-Location URI mit ein wenig Code definieren. Wir wollen also auf den Body der vorherigen Aktion (Parse_JSON) zugreifen und aus der JSON die varable Operation-Location auslesen. Wenn du den fogenden Code in die HTTP GET Abfrage einfügst, kannst du dennoch die Variable definieren.

@{body('Parse_JSON')?['Operation-Location']}

Ich hoffe du kannst immernoch folgen und falls nicht kann ich dir nur empfehlen die Logik App einmal auszuführen und dir die Ausgaben der einzelnen Arbeitsschritte einmal genauer anzusehen. Das wir dir bestimmt zu einem besseren Verständniss verhelfen.

<img src="../media/LogicAppImages/HTTPGET.gif" width="1000">

## <font color='#0066cc'>Test der Computer Vision Logik</font>

Wenn du nun die Logik App erneut testest, solltest du nach der HTTP GET Abfrage einen Output mit verschiedenen "bounding-boxen" und den dazugehörigen Texten erhalten.

Falls nicht, helfe ich dir gerne!

***

## <font color='#0066cc'>Erweitern der Logik für die Auswahl der besten Objekterkennungs Vorhersagen</font>

Da wir nun auch eine Ausgabe für die Erkennung der Zahlen und Buchstaben erhalten, können wir damit beginnen die beiden Ergebnisse miteinander zu verbinden. Zu erst müssen wir für unsere Objekterkennung die Recheckbox mit der höchsten Vorhersagewahrscheinlichkeit herausfinden. Genau dieses Recheck sollten, wenn das Modell richtig liegt, unser Zahlenbereich sein.

Dazu lernen wir einige neue Element der Logic Apps kennen:

- Variabeln: Damit können wir Ausgaben mit verschiedenen Datentypen speichern und wieder aufrufen
- ForEach Schleifen: Mit Schleifen können wir durch alle Elemente eines Array iterieren und auf die Elemente Operationen durchführen
- Append to Array Vaiable: Wir können an eine bestehende Vaiable einen neuen Wert anhängen.

Mit diesen Elementen könnn wir die beste Vorhersage aus unserem Objekterkennungsergebnisse herausfiltern.

- Erstellen einer Array Variable in der alle Modell Vorhersagen über 0.8 gespeichert werden
- Erstellen eine Foreach loops, um durch alle Vorhersagen des Objekterkennung Modells zu iterieren
- Parse JSON für jedes ForEachLoop item, damit wir auf die Genauigkeiten zugreifen können
- Einfügen einer Condition bei der wir alle Genauigkeiten mit einem Wert von größer als 0.8 nach true sortieren


##### <font color='#0066cc'>Erstellen einer Array Variable</font>

<img src="../media/LogicAppImages/init_var.gif" width="1000">

##### <font color='#0066cc'>Erweitern der Logik für die Auswahl der besten Objekterkennungs Vorhersagen</font>

Hier ist das benötigte Json Shema. Wie immer könnt ihr euch es auch selbst erstellen lassen wenn ihr den Output von Custom Vison anschaut bzw. die Option in dem Parser verwendet. 

In [3]:
{
    "properties": {
        "boundingBox": {
            "properties": {
                "height": {
                    "type": "number"
                },
                "left": {
                    "type": "number"
                },
                "top": {
                    "type": "number"
                },
                "width": {
                    "type": "number"
                }
            },
            "type": "object"
        },
        "probability": {
            "type": "number"
        },
        "tagId": {
            "type": "string"
        },
        "tagName": {
            "type": "string"
        }
    },
    "type": "object"
}

{'properties': {'boundingBox': {'properties': {'height': {'type': 'number'},
    'left': {'type': 'number'},
    'top': {'type': 'number'},
    'width': {'type': 'number'}},
   'type': 'object'},
  'probability': {'type': 'number'},
  'tagId': {'type': 'string'},
  'tagName': {'type': 'string'}},
 'type': 'object'}

<img src="../media/LogicAppImages/for_each_parse.gif" width="1000">

##### <font color='#0066cc'>Erweitern der Logik für die Auswahl der besten Objekterkennungs Vorhersagen</font>

<img src="../media/LogicAppImages/condition.gif" width="1000">

##### <font color='#0066cc'>Testen der eingefügten Logik</font>

Achtet beim testen der Logik darauf, dass ihr die einzelnen Elemente der ForEachScheife durchgehen könnt. Wundert euch also nicht, wenn in der variable zunächst kein Wert erscheint.

## <font color='#0066cc'>Parsen der Computer Vision Json ausgaben</font>

Im nächsten Schritt müssen wir die Json Ausgaben Parsen damit wir diese weiter verarbeiten können. Dazu müsse wir das Shema der Computer Vision Ausgaben definieren. Es Spart ein wenig Zeit, wenn man die Logik App einmal laufen lässt und dann die Ausgabe von dem HTTP Trigger anzeigen lässt. Du musst dann nurnoch den output kopieren und die richtigen Daten Typen definieren.


<img src="../media/LogicAppImages/jsonparse.gif" width="1000">

***

## <font color='#0066cc'>Zusammen führen der Json ausgaben und Speichern der Ergebnisse</font>

***

## <font color='#0066cc'>Nächsten Schritte</font>